In [1]:
import numpy as np
from RegressionFunc import *

In [2]:
df = pd.read_csv('../data/gfs_days.csv', sep = ';', index_col = 'Date', parse_dates = ['Date'])

# Температурные инверсии
df['DT1'] = df['TMP_mb_1000'] - df['TMP_mb_925']
df['DT2'] = df['TMP_mb_925'] - df['TMP_mb_850']
df['DT3'] = df['TMP_mb_1000'] - df['TMP_mb_850']

### Температура  на разных слоях, высота планетарного пограничного слоя, инверсии

In [3]:
df = df[[feat for feat in df.columns if "TMP_" in feat] + ['HPBL', 'DT1', 'DT2', 'DT3']] 
df

,TMP_m_2,TMP_m_80,TMP_m_100,TMP_mb_700,TMP_mb_750,TMP_mb_800,TMP_mb_850,TMP_mb_900,TMP_mb_925,TMP_mb_950,...,TMP_msl_2743,TMP_msl_3658,TMP_pv_p2,TMP_sig_0995,TMP_surf,TMP_tropo,HPBL,DT1,DT2,DT3
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,-15.036028,-14.065619,-14.010622,-20.649996,-18.626736,-17.012074,-15.319868,-13.626802,-12.756472,-12.964930,...,-19.784126,-22.812513,-62.409014,-14.221346,-16.305109,-62.673685,43.268045,-1.086754,2.563396,1.476642
2019-01-02,-15.554727,-14.968962,-14.905069,-23.507378,-20.651045,-18.688894,-17.069113,-15.133181,-14.224478,-13.897070,...,-22.252352,-27.242889,-56.770241,-15.086241,-16.463743,-59.259321,97.546916,-0.483261,2.844635,2.361374
2019-01-03,-17.415102,-16.447024,-16.394486,-25.180206,-22.337277,-20.026792,-18.559624,-17.449488,-16.754423,-16.326780,...,-24.025007,-27.632092,-55.220438,-16.639517,-18.737488,-59.174357,85.520970,0.401894,1.805201,2.207094
2019-01-04,-19.989693,-18.020252,-17.768793,-19.825367,-18.607857,-17.459810,-16.329878,-14.933958,-14.451819,-15.250547,...,-19.354433,-22.188843,-62.408976,-18.575432,-21.945013,-62.895603,40.738016,-3.801737,1.878059,-1.923678
2019-01-05,-18.821886,-18.110851,-18.075773,-21.143309,-18.679708,-16.926903,-15.861925,-15.289194,-15.490312,-16.855044,...,-20.236590,-25.053589,-60.781376,-18.221583,-20.136263,-61.614156,263.549033,-2.246016,0.371612,-1.874403
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-23,-7.618063,-4.470061,-4.146880,-14.083689,-12.260530,-10.486384,-9.124845,-6.647333,-5.022295,-3.504878,...,-13.467420,-17.252581,-60.608404,-5.325735,-8.122816,-61.610197,281.907345,-1.346138,4.102550,2.756412
2023-03-24,-3.271002,-2.582552,-2.553904,-14.877546,-13.351339,-11.571100,-9.495936,-6.107657,-4.617419,-3.169116,...,-14.328211,-17.802200,-53.467092,-2.680350,-3.933524,-58.469564,437.416414,2.578205,4.878517,7.456722
2023-03-25,0.085931,0.274252,0.151678,-10.579813,-8.487696,-7.807974,-6.302184,-4.067301,-2.481543,-0.928030,...,-9.755892,-14.163015,-64.430212,0.515627,-1.394549,-64.706514,687.471212,3.989250,3.820641,7.809891


### Давление, влажность, направление и скорость ветра

In [4]:
sensor = 's'
fill = '_fill'

district = 'Mean'

# Данные pm за 4+ года
df['pm'] = pd.read_csv(f"../data/pm25_{sensor}{fill}.csv", sep = ';', index_col = ['Date'], parse_dates = ['Date'], usecols = ['Date', district])[:"2023-03-27"].values

features = ['Pressure', 'Wet', 'Wind_dir', 'Wind_speed']
for feat in features:
    df[feat] = pd.read_csv(f"../data/Features{fill}/{feat}_{sensor}.csv", sep = ';', index_col = ['Date'], parse_dates = ['Date'], usecols = ['Date', district])[:"2023-03-27"]
df

,TMP_m_2,TMP_m_80,TMP_m_100,TMP_mb_700,TMP_mb_750,TMP_mb_800,TMP_mb_850,TMP_mb_900,TMP_mb_925,TMP_mb_950,...,TMP_tropo,HPBL,DT1,DT2,DT3,pm,Pressure,Wet,Wind_dir,Wind_speed
Date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,-15.036028,-14.065619,-14.010622,-20.649996,-18.626736,-17.012074,-15.319868,-13.626802,-12.756472,-12.964930,...,-62.673685,43.268045,-1.086754,2.563396,1.476642,220.8950,761.25,63.475,237.00,0.70
2019-01-02,-15.554727,-14.968962,-14.905069,-23.507378,-20.651045,-18.688894,-17.069113,-15.133181,-14.224478,-13.897070,...,-59.259321,97.546916,-0.483261,2.844635,2.361374,103.4725,761.50,64.950,270.00,1.70
2019-01-03,-17.415102,-16.447024,-16.394486,-25.180206,-22.337277,-20.026792,-18.559624,-17.449488,-16.754423,-16.326780,...,-59.174357,85.520970,0.401894,1.805201,2.207094,20.8300,762.75,63.900,273.00,2.80
2019-01-04,-19.989693,-18.020252,-17.768793,-19.825367,-18.607857,-17.459810,-16.329878,-14.933958,-14.451819,-15.250547,...,-62.895603,40.738016,-3.801737,1.878059,-1.923678,133.9000,761.50,62.750,243.00,1.30
2019-01-05,-18.821886,-18.110851,-18.075773,-21.143309,-18.679708,-16.926903,-15.861925,-15.289194,-15.490312,-16.855044,...,-61.614156,263.549033,-2.246016,0.371612,-1.874403,119.5750,757.50,62.125,268.00,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-23,-7.618063,-4.470061,-4.146880,-14.083689,-12.260530,-10.486384,-9.124845,-6.647333,-5.022295,-3.504878,...,-61.610197,281.907345,-1.346138,4.102550,2.756412,24.9400,753.00,52.175,276.54,1.45
2023-03-24,-3.271002,-2.582552,-2.553904,-14.877546,-13.351339,-11.571100,-9.495936,-6.107657,-4.617419,-3.169116,...,-58.469564,437.416414,2.578205,4.878517,7.456722,15.2200,748.25,57.875,247.66,2.46
2023-03-25,0.085931,0.274252,0.151678,-10.579813,-8.487696,-7.807974,-6.302184,-4.067301,-2.481543,-0.928030,...,-64.706514,687.471212,3.989250,3.820641,7.809891,6.2400,747.75,51.100,220.05,4.23


In [5]:
df.dropna(inplace = True)
df.shape

(1546, 30)

# Линейная регрессия

## Выборки по периодам повышенной концентрации pm 2.5

In [6]:
# Разделение данных по сезонам
def splitDataBySeason(df, season):
    # Зима
    if season == 'winters':
        win19 = df[:'2019-03-01 00:00:00']
        win19_20 = df['2019-11-27 00:00:00':'2020-02-27 00:00:00']
        win20_21 = df['2020-11-27 00:00:00':'2021-02-25 00:00:00']
        win21_22 = df['2021-12-15 00:00:00':'2022-02-22 00:00:00']
        win22_23 = df['2022-12-13 00:00:00' : '2023-02-22 00:00:00']

        return pd.concat([win19, win19_20, win20_21, win21_22, win22_23])
    
    # Весна
    if season == 'springs':
        spr19 = pd.concat([df['2019-03-10' : '2019-05-01'], df['2019-11-01' : '2019-11-22']])
        spr20 = pd.concat([df['2020-03-01' : '2020-05-01'], df['2020-11-01' : '2020-11-10']])
        spr21 = pd.concat([df['2021-03-10' : '2021-05-01'], df['2021-11-01' : '2021-11-25']])
        spr22 = pd.concat([df['2022-03-01' : '2022-05-01'], df['2022-11-01' : '2022-11-25']])
        spr23 = df['2023-03-01' : ]

        return pd.concat([spr19, spr20, spr21, spr22, spr23])

    # Лето
    if season == 'summers':
        sum19 = df['2019-05-01' : '2019-07-12']
        sum20 = df['2020-05-01' : '2020-08-01']
        sum21 = df['2021-05-01' : '2021-08-01']
        sum22 = df['2022-05-01' : '2022-08-01']

        return pd.concat([sum19, sum20, sum21, sum22])
    
    # Осень
    if season == 'autumns':
        aut19 = df['2019-08-21' : '2019-10-16']
        aut20 = df['2020-08-17' : '2020-10-30']
        aut21 = df['2021-08-15' : '2021-11-01']
        aut22 = df['2022-08-01' : '2022-10-14']

        return pd.concat([aut19, aut20, aut21, aut22])

## Отбор признаков, коррелирующих с pm >= 0.2

In [7]:
pd.options.mode.chained_assignment = None

# Сезон

In [8]:
season = 'autumns'

In [9]:
data = splitDataBySeason(df, season)

### Построение моделей

In [10]:
resultDfs = BuildModels(data)

,Train,linear,lasso,ridge,Полиномиальная 2 степени,Полиномиальная 3 степени
0,MSE,13.86,24.94,14.24,0.0,0.0
1,MAE,2.85,3.62,2.87,0.0,0.0
2,MAPE,0.37,0.45,0.37,0.0,0.0
3,R2,0.74,0.54,0.74,1.0,1.0
4,R2_adj,0.70,0.46,0.69,1.0,1.0


,Valid,linear,lasso,ridge,Полиномиальная 2 степени,Полиномиальная 3 степени
0,MSE,33.77,41.06,32.04,317.39,146.85
1,MAE,3.73,4.12,3.62,11.83,8.86
2,MAPE,0.36,0.38,0.34,1.05,1.18
3,R2,0.58,0.49,0.60,-2.91,-0.81
4,R2_adj,0.12,-0.07,0.16,1.53,1.02


,Test,linear,lasso,ridge,Полиномиальная 2 степени,Полиномиальная 3 степени
0,MSE,15.61,27.52,16.41,187.78,225.94
1,MAE,2.98,3.95,3.17,10.16,11.44
2,MAPE,0.41,0.53,0.43,1.62,2.09
3,R2,0.65,0.38,0.63,-3.21,-4.07
4,R2_adj,10.81,18.29,11.31,1.27,1.03


### Отбор признаков